In [1]:
import numpy as np
import math
import ipywidgets as widgets
from ipycanvas import Canvas, hold_canvas
from time import sleep

In [2]:
def draw_seg(canvas:Canvas, v1, v2):
    if any(np.isnan(v1)) or any(np.isnan(v2)):
        return
    canvas.stroke_line(
        int(v1[0]), 
        int(v1[1]), 
        int(v2[0]), 
        int(v2[1]))

def draw_vertices(canvas:Canvas, vertices):
    for i in range(1, len(vertices)):
        draw_seg(canvas, vertices[i-1], vertices[i])
        
def rotate_x(vertices, a):
    m = np.array([
        [1, 0, 0, 0],
        [0, math.cos(a), math.sin(a), 0],
        [0, -math.sin(a), math.cos(a), 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

def rotate_y(vertices, a):
    m = np.array([
        [math.cos(a), 0, -math.sin(a), 0],
        [0, 1, 0, 0],
        [math.sin(a), 0, math.cos(a), 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

def rotate_z(vertices, a):
    m = np.array([
        [math.cos(a), math.sin(a), 0, 0],
        [-math.sin(a), math.cos(a), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

def translate(vertices, dx, dy, dz):
    m = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [dx, dy, dz, 1],
    ])
    return vertices @ m

def scale(vertices, sx, sy, sz):
    m = np.array([
        [sx, 0, 0, 0],
        [0, sy, 0, 0],
        [0, 0, sz, 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

In [3]:
CANVAS_HEIGTH = 400
CANVAS_WIDTH = 800

In [4]:
# x y z h, p, y
class Camera:
    def __init__(self, position, heading, pitch, yaw):
        self.position = position
        self.heading = heading
        self.pitch = pitch
        self.yaw = yaw
    def rotate_matrix(self):
        return rotate_z(rotate_x(rotate_y(np.eye(4), -self.heading), -self.pitch), -self.yaw)
    def translate_matrix(self):
        return translate(np.eye(4), -self.position[0], -self.position[1], -self.position[2])
    def view_matrix(self):
        return self.translate_matrix() @ self.rotate_matrix()
camera = Camera([-5,6,-300], math.radians(0),math.radians(0),math.radians(0))
camera.view_matrix()

array([[  1.,   0.,   0.,   0.],
       [  0.,   1.,   0.,   0.],
       [  0.,   0.,   1.,   0.],
       [  5.,  -6., 300.,   1.]])

In [5]:
h_fov = math.pi / 3
v_fov = h_fov * (CANVAS_HEIGTH / CANVAS_WIDTH)

NEAR = 0.1
FAR = 100
RIGHT = math.tan(h_fov / 2)
LEFT = -RIGHT
TOP = math.tan(v_fov / 2)
BOTTOM = -TOP

m00 = 2.0 / (RIGHT - LEFT)
m11 = 2.0 / (TOP - BOTTOM)
m22 = (FAR + NEAR)*1.0 / (FAR - NEAR)
m32 = -2.0 * NEAR * FAR / (FAR - NEAR)

projection_matrix = np.array([
    [m00, 0, 0, 0],
    [0, m11, 0, 0],
    [0, 0, m22, 1],
    [0, 0, m32, 0]
])
projection_matrix

array([[ 1.73205081,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  3.73205081,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.002002  ,  1.        ],
       [ 0.        ,  0.        , -0.2002002 ,  0.        ]])

In [6]:
HW, HH = CANVAS_WIDTH/2, CANVAS_HEIGTH/2
to_screen_matrix = np.array([
    [HW, 0, 0, 0],
    [0, -HH, 0, 0],
    [0, 0, 1, 0],
    [HW, HH, 0, 1]
])
to_screen_matrix

array([[ 400.,    0.,    0.,    0.],
       [   0., -200.,    0.,    0.],
       [   0.,    0.,    1.,    0.],
       [ 400.,  200.,    0.,    1.]])

In [7]:
obj_cube_vertices = np.array([
    [-50, -50, -50, 1],
    [-50,  50, -50, 1],
    [ 50,  50, -50, 1],
    [ 50, -50, -50, 1],
    [-50, -50, 50, 1], 
    [-50,  50, 50, 1], 
    [ 50,  50, 50, 1], 
    [ 50, -50, 50, 1]] 
)
obj_cube_seg_indices = np.array([
    0,1,1,2,2,3,3,0,
    4,5,5,6,6,7,7,4,
    0,4,1,5,2,6,3,7
])

def draw_segments_element(canvas:Canvas, vertices, indices):
    for i in range(0, len(indices), 2):
        draw_seg(canvas, vertices[indices[i]], vertices[indices[i+1]])


In [8]:
def to_screen(vercies):
    obj = vercies
    obj = obj @ camera.view_matrix()
    obj = obj @ projection_matrix
    # 这里使用w值归一化
    obj /= obj[:, -1].reshape(-1, 1)
    obj[(obj > 2) | (obj < -2)] = None
    obj = obj @ to_screen_matrix
    obj = obj[:,:2]
    return obj

# 相机+投影变换

In [9]:
canvas = Canvas(height=CANVAS_HEIGTH, width=CANVAS_WIDTH)
display(canvas)

def render():
    with hold_canvas():
        canvas.clear()
        draw_segments_element(canvas, to_screen(obj_cube_vertices), obj_cube_seg_indices)
render()

headingSlider = widgets.FloatSlider(value=camera.heading, min=0, max=360, description='heading:', layout=widgets.Layout(width='500px'))
def headingChanged(evt):
    camera.heading = math.radians(evt.new)
    render()
headingSlider.observe(headingChanged, 'value')

pitchSlider = widgets.FloatSlider(value=camera.pitch, min=0, max=360, description='pitch:', layout=widgets.Layout(width='500px'))
def pitchChanged(evt):
    camera.pitch = math.radians(evt.new)
    render()
pitchSlider.observe(pitchChanged, 'value')

yawSlider = widgets.FloatSlider(value=camera.yaw, min=0, max=360, description='yaw:', layout=widgets.Layout(width='500px'))
def yewChanged(evt):
    camera.yaw = math.radians(evt.new)
    render()
yawSlider.observe(yewChanged, 'value')

xSlider = widgets.FloatSlider(value=camera.position[0], min=-400, max=400, description='x:', layout=widgets.Layout(width='500px'))
def xChanged(evt):
    camera.position[0] = evt.new
    render()
xSlider.observe(xChanged, 'value')

ySlider = widgets.FloatSlider(value=camera.position[1], min=-400, max=400, description='y:', layout=widgets.Layout(width='500px'))
def yChanged(evt):
    camera.position[1] = evt.new
    render()
ySlider.observe(yChanged, 'value')

zSlider = widgets.FloatSlider(value=camera.position[2], min=-400, max=400, description='z:', layout=widgets.Layout(width='500px'))
def zChanged(evt):
    camera.position[2] = evt.new
    render()
zSlider.observe(zChanged, 'value')

widgets.VBox([headingSlider, pitchSlider, yawSlider, xSlider, ySlider, zSlider])

Canvas(height=400, width=800)